**INSTALAMOS LIBRERIAS PARA CARGA**

In [ ]:
! pip install psycopg2
! pip install pgdb 

     |████████████████████████████████| 3.0 MB 14.9 MB/s 
  Created wheel for pgdb: filename=pgdb-0.0.11-py3-none-any.whl size=2800 sha256=f70005daee79ed0cc1ac7c92a42e5f780e2baacd3e7ce3fc9d875ee89b3ac386
  Stored in directory: /root/.cache/pip/wheels/ff/83/6a/8cd1726f08b2d24a023f99cdb271b156b9d61ab3771ed80e30
Successfully built pgdb


**ASIGNAMOS EL PROYECTO DONDE SE ENCUENTRA NUESTRA BD Y EL BUCKET DE LOS DATOS QUE SE CARGARAN**

In [ ]:
PROJECT_ID = "keepcoding-proyectofinal-01" #@param {type:"string"}
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [ ]:
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

if 'google.colab' in sys.modules:
  from google.colab import auth as google_auth
  google_auth.authenticate_user()

# If you are running this notebook locally, replace the string below with the
# path to your service account key and run this cell to authenticate your GCP
# account.
else:
  %env GOOGLE_APPLICATION_CREDENTIALS ''

In [ ]:
BUCKET_NAME = "kcp-bucket-01" #@param {type:"string"}
REGION = "europe-west1" #@param {type:"string"}

In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

                                 gs://kcp-bucket-01/datos_detection/
                                 gs://kcp-bucket-01/datos_equivalencies/
                                 gs://kcp-bucket-01/datos_images_out/
                                 gs://kcp-bucket-01/datos_shopping_list/
                                 gs://kcp-bucket-01/datos_store/


**COPIAMOS LA CARPETA DATOS DE NUESTRO BUCKET A NUESTRO LOCAL PARA TRATARLOS ANTES DE REALIZAR LA CARGA**

In [ ]:
! gsutil cp -r gs://$BUCKET_NAME/datos_equivalencies /content/

Copying gs://kcp-bucket-01/datos_equivalencies/Datos_Equivalencies_20220328.csv...
/ [1 files][  213.0 B/  213.0 B]                                                
Operation completed over 1 objects/213.0 B.                                      


LEEMOS EL FICHERO CSV DE EQUIVALENCIES A UN DATAFRAME

In [ ]:
import pandas as pd

df_data_equivalencies = pd.read_csv('/content/datos_equivalencies/Datos_Equivalencies_20220328.csv', index_col=None)

ASIGNAMOS LA FECHA DEL DIA AL CAMPO DATE_CREATED (FECHA DE CREACION)

In [ ]:
from datetime import datetime

df_data_equivalencies['date_created'] = datetime.today()

COMPROBAMOS LOS DATOS DEL DATAFRAME

In [ ]:
print(df_data_equivalencies.columns)
df_data_equivalencies.head()

Index(['date_created', 'product', 'qty_kg', 'qty_pieces'], dtype='object')


,date_created,product,qty_kg,qty_pieces
0,2022-03-31 22:28:36.889385,lemon,1,10
1,2022-03-31 22:28:36.889385,apple,1,6
2,2022-03-31 22:28:36.889385,bananas,1,5
3,2022-03-31 22:28:36.889385,carrot,1,8
4,2022-03-31 22:28:36.889385,orange,1,6


GENERAMOS UN NUEVO CSV CON LOS DATOS CON EL NUEVO FORMATO. APROVECHAMOS PARA QUITAR EL INDICE.

In [ ]:
df_data_equivalencies.to_csv('/content/datos_equivalencies/Datos_Equivalencies_format.csv', index=False, sep=',')

**IMPORTAMOS LIBRERIAS PARA TRABAJAR CON LA CARGA DE DATOS**



In [ ]:
import psycopg2
import pgdb

**CARGA DE DATOS EN LA TABLA EQUIVALENCIES**

In [ ]:
def cargar(path):
    """ Conexión al servidor de pases de datos PostgreSQL """
    conexion = None
    try:
        # Lectura de los parámetros de conexion
        #params = config()
 
        # Conexion al servidor de PostgreSQL
        print('Conectando a la base de datos PostgreSQL...')
        conexion = pgdb.Connection(host="34.79.94.25",database="proyecto", user="postgres", password="keepcoding")
 
        # creación del cursor
        cur = conexion.cursor()

        csv_file_name = path
        sql = "COPY Equivalencies FROM STDIN DELIMITER ',' CSV HEADER"
        cur.copy_expert(sql, open(csv_file_name, "r"))
        conexion.commit()

        # Cierre de la comunicación con PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conexion is not None:
            conexion.close()
            print('Conexión finalizada.')

In [ ]:
cargar('/content/datos_equivalencies/Datos_Equivalencies_format.csv')

Conectando a la base de datos PostgreSQL...
Conexión finalizada.
